In [4]:
#Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
#Carga desde un archivo Excel omitiendo las primeras 4 filas
data = pd.read_csv(r"C:\Users\navae\OneDrive\Escritorio\Tec\Semestre 5\Plataformas de analítica de negocios para organizaciones\Clonación Github\Valores_Nulos\Actividad 3.3\Italia_filtrado_es.csv")
data.head(5)

,id,id_anfitrion,ultima_extraccion,zona_grupo,vecindario,latitud,longitud,tipo_propiedad,tipo_habitacion,capacidad,...,tasa_aceptacion_host,host_tiene_foto,host_verificado,antiguedad_host,vecindario_host,total_listados_host,host_total_casas_completas,host_total_habitaciones_privadas,host_total_habitaciones_compartidas,licencia
0,44527,120215,2025-06-17,Isole,Cannaregio,45.44569,12.32902,Entire rental unit,Entire home/apt,4,...,100%,t,t,2010-05-07,Cannaregio,2,2,0,0,IT027042B4GR0PSZS4
1,47383,214390,2025-06-17,Isole,San Polo,45.43921,12.33310,Room in bed and breakfast,Hotel room,2,...,100%,t,t,2010-08-27,San Polo,4,1,0,0,IT027042B4D3D4AUUS
2,49656,226598,2025-06-17,Isole,Dorsoduro,45.43549,12.32612,Entire rental unit,Entire home/apt,2,...,100%,t,t,2010-09-07,Dorsoduro/Accademia,2,2,0,0,IT027042C2L6NZYNJR
3,60304,290452,2025-06-17,Isole,Cannaregio,45.44453,12.33300,Private room in bed and breakfast,Private room,2,...,75%,t,t,2010-11-18,Cannaregio,2,0,2,0,IT027042B4JG3TJH56
4,61714,289023,2025-06-17,Isole,Giudecca,45.42708,12.31823,Private room in bed and breakfast,Private room,2,...,100%,t,t,2010-11-16,Giudecca,2,0,2,0,IT027042C1NT8RRIFS


In [6]:
#Exploración inicial de los datos
print("Dimensiones:", data.shape)
print("\nTipos de datos:")
print(data.dtypes)
print("\nValores nulos por columna:")
print(data.isnull().sum())

Dimensiones: (8250, 54)

Tipos de datos:
id                                       int64
id_anfitrion                             int64
ultima_extraccion                       object
zona_grupo                              object
vecindario                              object
latitud                                float64
longitud                               float64
tipo_propiedad                          object
tipo_habitacion                         object
capacidad                                int64
banos                                  float64
recamaras                              float64
camas                                  float64
noches_minimas                           int64
noches_maximas                           int64
min_noches_minimas                       int64
max_noches_minimas                       int64
min_noches_maximas                       int64
max_noches_maximas                       int64
precio                                  object
tiene_disponibilida